In [1]:
# %%writefile ./learning_model/train.py
"""
Created on Mon Apr 19 05:57:12 2021
@author: Hwarang Kim

The task is to predict if members will take learning courses in 2021Q2
based on last three year learning behaviour and profiles
"""

# =========================
# 1. Setup & load dataset
# ========================
# a) Load libraries
# python libs
import numpy as np
import pandas as pd
from pandas import read_csv
from pandas import set_option

# data transformation
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

# classification models 
from sklearn.ensemble import GradientBoostingClassifier

# b) Load dataset from bigqquery table
from google.cloud import bigquery
client = bigquery.Client(location="US")
query = "SELECT * FROM `scs-3760-hkim.term_project.v_dim_learner`"
query_job = client.query(query, location="US")
loaded = query_job.to_dataframe()

# ================
# 2. Analyze Data
# ================
# handling missing value
dataset = loaded.fillna(0)

# ================
# 3. Prepare Data
# ================
# generate label and features
# label; featCat; featNum
labelCol = ['learning_2021Q1']
featCatCol = ['gender', 'empl_type', 'emp_industry', 'fee_waiver', 'prof_group', 'ont', 'gta','life', 'mentor','age','tenure']
featNumCol = ['c_level', 'emp_size', 'avg_income', 'cnt_learning',
               'learning_2018Q1', 'learning_2018Q2', 'learning_2018Q3', 'learning_2018Q4',
               'learning_2019Q1', 'learning_2019Q2', 'learning_2019Q3', 'learning_2019Q4',
               'learning_2020Q1', 'learning_2020Q2', 'learning_2020Q3', 'learning_2020Q4']

label = dataset[labelCol]
featCat = dataset[featCatCol]
featNum = dataset[featNumCol]

# label
enc = LabelEncoder() # expect D1 - List
labelEnc = enc.fit_transform(label.values.ravel())

# featNum
mm = MinMaxScaler()
featNumEnc = mm.fit_transform(featNum)

# featCat OrdinalEncoder
enc = OrdinalEncoder()
featCatOrdEnc = enc.fit_transform(featCat)

# featOreEnc = featCatOrdEnc + featNumEnc
featOreEnc = np.concatenate((featCatOrdEnc,featNumEnc), axis=1)

# convert to dataframe
df_label = pd.DataFrame(labelEnc, columns=['label'])
df_featOre = pd.DataFrame(featOreEnc)
df_featOre_set = pd.concat([df_label, df_featOre], axis=1)

# ======================================
# 5. Train the model
# ======================================
# Tune RandomForestClassifier(RF)
X_train = df_featOre_set.drop(['label'], axis=1)
Y_train = df_featOre_set['label']

model = GradientBoostingClassifier(learning_rate=0.5, max_depth=3, n_estimators=200)
model.fit(X_train, Y_train)

# ============================
# 6. Data Prep for Prediction
# ============================

# generate label and features
# featCat; featNum
featCatCol = ['gender', 'empl_type', 'emp_industry', 'fee_waiver', 'prof_group', 'ont', 'gta','life', 'mentor','age','tenure']
featNumCol = [
    'c_level', 'emp_size', 'avg_income', 'cnt_learning',
    #'learning_2018Q1', 
    'learning_2018Q2', 'learning_2018Q3', 'learning_2018Q4', 'learning_2019Q1', 
    'learning_2019Q2', 'learning_2019Q3', 'learning_2019Q4', 'learning_2020Q1', 
    'learning_2020Q2', 'learning_2020Q3', 'learning_2020Q4', 'learning_2021Q1']

featCat = dataset[featCatCol]
featNum = dataset[featNumCol]

# featNum
mm = MinMaxScaler()
featNumEnc = mm.fit_transform(featNum)

# featCat OrdinalEncoder
enc = OrdinalEncoder()
featCatOrdEnc = enc.fit_transform(featCat)

# featOreEnc = featCatOrdEnc + featNumEnc
featOreEnc = np.concatenate((featCatOrdEnc,featNumEnc), axis=1)

# convert to dataframe
df_featOre = pd.DataFrame(featOreEnc)

# =====================================
# 7. Predict Learning for Next Quarter
# =====================================
Y_pred = model.predict(df_featOre)
df_Y_pred = pd.concat([loaded["learner_id"],pd.DataFrame({'pred':Y_pred})], axis=1)

# ================================
# 8. Load Prediction to biqquery
# ===============================
# drop table learning_pred
sql = "drop table if exists `scs-3760-hkim.term_project.learning_pred`"
client.query(sql, location="US")

# load
bg_dataset = client.dataset('term_project')
table_ref = bg_dataset.table("learning_pred")
job = client.load_table_from_dataframe(df_Y_pred, table_ref, location="US")

job.result()  # Waits for table load to complete.
print("Loaded dataframe to {}".format(table_ref.path))


Loaded dataframe to /projects/scs-3760-hkim/datasets/term_project/tables/learning_pred
